In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm

/tmp/ipykernel_4567/234451680.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
all_data = pd.read_csv('./FedUL/data/ausgrid/2011_2013-anomaly_labeled_dataset.csv', sep=',', index_col='index')
clients = all_data['costumer'].unique().tolist()

In [3]:
all_data.head(10)

,costumer,GC,hourofday,minuteofhour,dayofweek,dayofmonth,monthofyear,year,Anomaly
index,,,,,,,,,
0,7,0.580,0,0,4,1,7,2011,0
1,7,0.083,0,30,4,1,7,2011,0
2,7,0.156,1,0,4,1,7,2011,0
3,7,0.097,1,30,4,1,7,2011,0
4,7,0.058,2,0,4,1,7,2011,0
5,7,0.179,2,30,4,1,7,2011,0
6,7,0.095,3,0,4,1,7,2011,0
7,7,0.097,3,30,4,1,7,2011,0
8,7,0.120,4,0,4,1,7,2011,0


In [4]:
all_data.keys()

Index(['costumer', 'GC', 'hourofday', 'minuteofhour', 'dayofweek',
       'dayofmonth', 'monthofyear', 'year', 'Anomaly'],
      dtype='object')

In [5]:
clients

[7, 29, 30, 64, 155, 160, 184, 202, 206, 215]

In [6]:
client_labels = all_data[all_data['costumer']==clients[0]]['Anomaly'].values
client_features = all_data[all_data['costumer']==clients[0]].iloc[:, :-1].values

In [7]:
# gerando um np.array com label e feature de cada cliente com novos indices
for idx, client in enumerate(tqdm(clients)):
    
    client_labels = all_data[all_data['costumer']==client]['Anomaly'].values
    # drop anomaly columns and costumer column
    client_features = all_data[all_data['costumer']==client].iloc[:, 1:-1].values


    # split train and test(20%) for each client, NO SHUFFLE to avoid Time Leakage
    x_train_temp, x_test, y_train_temp, y_test = train_test_split(client_features, client_labels, test_size=0.2, stratify=client_labels, random_state=42, shuffle=False)
    
    # split train into train and validation (10%) for each client, NO SHUFFLE to avoid Time Leakage
    x_train, x_val, y_train, y_val = train_test_split(x_train_temp, y_train_temp, test_size=0.1, stratify=y_train_temp, random_state=42, shuffle=False)

    ### transform to DataFrame

    # train
    train_table_client = pd.DataFrame.from_records(x_train)
    
    # val
    val_table_client = pd.DataFrame.from_records(x_val)
    val_labels_table_client = pd.DataFrame.from_records(y_val.reshape(-1,1))

    # test
    test_table_client = pd.DataFrame.from_records(x_test)
    test_labels_table_client = pd.DataFrame.from_records(y_test.reshape(-1,1))


    ### create CSV from client table
    # train
    train_table_client.to_csv(f'./split_dataset/train_{idx}.csv', index=False)
    
    # val
    val_table_client.to_csv(f'./split_dataset/val_{idx}.csv', index=False)
    test_labels_table_client.to_csv(f'./split_dataset/val_label_{idx}.csv', index=False)

    # test
    test_table_client.to_csv(f'./split_dataset/test_{idx}.csv', index=False)
    test_labels_table_client.to_csv(f'./split_dataset/test_label_{idx}.csv', index=False)
    

  0%|          | 0/10 [00:00<?, ?it/s]


ValueError: Stratified train/test split is not implemented for shuffle=False

In [6]:
client_labels

array([0, 0, 0, ..., 0, 0, 0])

In [7]:
client_features

array([[7.000e+00, 5.800e-01, 0.000e+00, ..., 1.000e+00, 7.000e+00,
        2.011e+03],
       [7.000e+00, 8.300e-02, 0.000e+00, ..., 1.000e+00, 7.000e+00,
        2.011e+03],
       [7.000e+00, 1.560e-01, 1.000e+00, ..., 1.000e+00, 7.000e+00,
        2.011e+03],
       ...,
       [7.000e+00, 7.500e-02, 2.200e+01, ..., 3.000e+01, 6.000e+00,
        2.013e+03],
       [7.000e+00, 8.700e-02, 2.300e+01, ..., 3.000e+01, 6.000e+00,
        2.013e+03],
       [7.000e+00, 1.180e-01, 2.300e+01, ..., 3.000e+01, 6.000e+00,
        2.013e+03]])

In [19]:
# split train and test(20%) for each client
x_train_temp, x_test, y_train_temp, y_test = train_test_split(client_features, client_labels, test_size=0.2, stratify=client_labels, random_state=42)

# split train into train and validation (10%) for each client
x_train, x_val, y_train, y_val = train_test_split(x_train_temp, y_train_temp, test_size=0.1, stratify=y_train_temp, random_state=42)


In [20]:
# transform to DataFrame
t = pd.DataFrame.from_records(y_val.reshape(-1,1))

In [22]:
g = pd.DataFrame.from_records(x_val)

In [21]:
t.shape

(2807, 1)

In [23]:
g.shape

(2807, 8)

In [37]:
# transformar em tabelas
train_table = pd.DataFrame.from_records(temp_x_train)
test_table = pd.DataFrame.from_records(temp_x_test)
test_labels_table = pd.DataFrame.from_records(temp_y_labels)

In [38]:
train_table.to_csv('train.csv', index=False)
test_table.to_csv('test.csv', index=False)
test_labels_table.to_csv('test_label.csv', index=False)

In [39]:
teste = pd.read_csv('train.csv')
teste

,0,1,2,3,4,5,6,7
0,7.0,0.196,15.0,0.0,5.0,17.0,3.0,2012.0
1,7.0,1.222,20.0,30.0,6.0,19.0,8.0,2012.0
2,7.0,0.261,20.0,0.0,3.0,30.0,8.0,2012.0
3,7.0,1.088,17.0,30.0,3.0,20.0,6.0,2013.0
4,7.0,0.388,9.0,30.0,6.0,9.0,6.0,2013.0
...,...,...,...,...,...,...,...,...
280695,215.0,0.328,20.0,30.0,0.0,9.0,4.0,2012.0
280696,215.0,1.274,13.0,0.0,5.0,2.0,6.0,2012.0
280697,215.0,0.057,16.0,30.0,3.0,16.0,8.0,2012.0
280698,215.0,0.183,12.0,0.0,3.0,24.0,11.0,2011.0
